### Import Necessary Packages

In [1]:
import pandas
import numpy as np
import os
import shutil
import torch

### Import Data

In [ ]:
labels = ["coca_cola","fanta","7up"]
for label in labels:
    os.makedirs(f'classification/train/{label}')
    os.makedirs(f'classification/val/{label}')
    os.makedirs(f'classification/test/{label}')

In [ ]:
cola_train_count = 0
cola_val_count = 0
cola_test_count = 0
fanta_train_count = 0
fanta_val_count = 0
fanta_test_count = 0
7up_train_count = 0
7up_val_count = 0
7up_test_count = 0

sub_dir = ["coca cola can", "coca cola plastic bottle", "fanta can", 
           "fanta plastic bottle", "7up can", "7up plastic bottle"]
for d in sub_dir:
    if d.split()[0] == "coca":
        for f in os.listdir(f'DATA/data/{d}'):
            if cola_train_count < 800:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/train/coca_cola/{cola_train_count}.jpg')
                cola_train_count += 1
            elif cola_val_count < 100:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/val/coca_cola/{cola_val_count}.jpg')
                cola_val_count += 1
            elif cola_test_count < 100:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/test/coca_cola/{cola_test_count}.jpg')
                cola_test_count += 1
    elif d.split()[0] == "fanta":
        for f in os.listdir(f'DATA/data/{d}'):
            if fanta_train_count < 800:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/train/fanta/{fanta_train_count}.jpg')
                fanta_train_count += 1
            elif fanta_val_count < 100:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/val/fanta/{fanta_val_count}.jpg')
                fanta_val_count += 1
            elif fanta_test_count < 100:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/test/fanta/{fanta_test_count}.jpg')
                fanta_test_count += 1
    elif d.split()[0] == "7up":
        for f in os.listdir(f'DATA/data/{d}'):
            if 7up_train_count < 800:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/train/7up/{7up_train_count}.jpg')
                7up_train_count += 1
            elif 7up_val_count < 100:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/val/7up/{7up_val_count}.jpg')
                7up_val_count += 1
            elif 7up_test_count < 100:
                shutil.copy(f'DATA/data/{d}/'+f,f'classification/test/7up/{7up_test_count}.jpg')
                7up_test_count += 1

In [2]:
train_dir = 'classification/train'
val_dir = 'classification/val'
test_dir = 'classification/test'

### Data Process

In [3]:
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms, datasets
from collections import OrderedDict

In [4]:
dataset_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.RandomRotation(30),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
# Create train, validation and test dataset
train_dataset = datasets.ImageFolder(train_dir,transform=dataset_transform)
val_dataset = datasets.ImageFolder(val_dir,transform=dataset_transform)
test_dataset = datasets.ImageFolder(test_dir,transform=dataset_transform)

In [6]:
# Load datasets into dataloader for iteration
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 100, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 100, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = True)

### Training process

In [7]:
# Training function
def training(model):
    iteration = 0
    minLoss = float("inf")
    best_parameters = None

    # Training Loop
    for epoch in range(epochs):
        # Iterate over the training dataloader
        for i, (images, labels) in enumerate(train_dataloader):
            # Reinitialize the gradient
            optimizer.zero_grad()

            # Get the correct label
            images, labels = images.to(device), labels.to(device)

            # Forward propagation
            outputs = model(images)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Use the loss for back propagation
            loss.backward()

            # Update network parameters
            optimizer.step()
            
            # Empty the cache of GPU
            del images
            del labels
            del loss
            del outputs
            torch.cuda.empty_cache()

            iteration += 1

            if iteration % 10 == 0 or iteration == 1:
                # Calculate Accuracy and loss        
                correct = 0
                total = 0

                total_loss = 0

                # Validation Loop
                # Iterate over the validation dataloader
                for images, labels in val_dataloader:

                    images, labels = images.to(device), labels.to(device)
                    # Make prediction
                    outputs = model(images)

                    loss = criterion(outputs, labels)
                    total_loss += float(loss)

                    # Get predictions from the maximum value, index 0 is the maximum value, 
                    # index 1 is the index of the maximum value
                    _, predicted = torch.max(outputs.data, 1)

                    # Number of labels
                    total += labels.size(0)

                    # Number of correct predictions
                    correct += (predicted == labels).sum()
                    
                    # Empty cache of GPU
                    del images
                    del labels
                    del loss
                    del outputs
                    torch.cuda.empty_cache()

                # Calculate accuracy
                accuracy = 100 * correct / total

                # Save parameters if loss is smaller
                if total_loss < minLoss:
                    minLoss = total_loss
                    best_parameters = model.state_dict()

                # Print accuracy and loss
                print(f'Iteration {iteration}: Loss: {total_loss}, Validation accuracy: {accuracy}')
    return best_parameters

In [8]:
import matplotlib.pyplot as plt
# Testing
def test(model):
    correct = 0
    total = 0
    total_loss = 0
    count = 0

    # Test Loop
    for images, labels in test_dataloader:

        images, labels = images.to(device), labels.to(device)

        # Make prediction
        outputs = model(images)

        loss = criterion(outputs, labels)
        total_loss += float(loss)

        # Get predictions from the maximum value, index 0 is the maximum value, 
        # index 1 is the index of the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Number of labels
        total += labels.size(0)

        # Number of correct predictions
        correct += (predicted == labels).sum()
        incorrect = images[predicted != labels]
        
        # Show 5 wrong predicted example
        if count < 5 and (predicted == labels).sum() != labels.size(0):
            
            wrong = "dog" if predicted[predicted != labels][0] else "cat"
            actual = "dog" if labels[predicted != labels][0] else "cat"
            print(f"Predicted {wrong}")
            print(f"Actual {actual}")
            show_image = incorrect[0].cpu().numpy().transpose((1, 2, 0))
            plt.imshow(show_image, cmap='gray')
            plt.show()
            count += 1
        
        # Empty cache
        del images
        del labels
        del loss
        del outputs
        torch.cuda.empty_cache()

    # Calculate accuracy
    accuracy = 100 * correct / total

    # Print accuracy and loss
    print(f'Best model has Loss: {total_loss}, Test accuracy: {accuracy}')

### VGG Model

In [58]:
VGG = models.vgg16(pretrained=True)
VGG

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [59]:
# Modify the last layer
VGG.classifier[6] = nn.Linear(in_features=4096, out_features=3)
# VGG.classifier.add_module("7", nn.Sigmoid())
# print(VGG.classifier)

In [60]:
# Only unfreeze the last two layer
to_update = ["classifier.6.weight", "classifier.6.bias"]
to_update_params = []
for name, param in VGG.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [61]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [62]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VGG = VGG.to(device)

In [63]:
# Training
vgg_best_parameters = training(VGG)

Iteration 1: Loss: 13.507441997528076, Validation accuracy: 46.333335876464844
Iteration 10: Loss: 5.346171617507935, Validation accuracy: 58.333335876464844
Iteration 20: Loss: 3.712480068206787, Validation accuracy: 70.0
Iteration 30: Loss: 3.6206597089767456, Validation accuracy: 68.0
Iteration 40: Loss: 3.330405592918396, Validation accuracy: 70.33333587646484
Iteration 50: Loss: 3.288989841938019, Validation accuracy: 71.66667175292969
Iteration 60: Loss: 2.475476086139679, Validation accuracy: 75.66667175292969
Iteration 70: Loss: 3.1271023750305176, Validation accuracy: 71.0
Iteration 80: Loss: 3.5054694414138794, Validation accuracy: 68.66667175292969
Iteration 90: Loss: 3.105809450149536, Validation accuracy: 70.0
Iteration 100: Loss: 2.715262234210968, Validation accuracy: 72.0
Iteration 110: Loss: 2.6015686988830566, Validation accuracy: 74.33333587646484
Iteration 120: Loss: 2.6435959339141846, Validation accuracy: 74.0
Iteration 130: Loss: 2.9155865907669067, Validation ac

### AlexNet Model

In [15]:
alexnet = models.alexnet(pretrained=True)
alexnet

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\Leo/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [16]:
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=3)

In [17]:
# Only unfreeze the last two layer
to_update = ["classifier.6.weight", "classifier.6.bias"]
to_update_params = []
for name, param in alexnet.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [20]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [21]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

In [22]:
# Training
alexnet_best_parameters = training(alexnet)

C:\Users\Leo\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Iteration 1: Loss: 7.932284593582153, Validation accuracy: 58.66666793823242
Iteration 10: Loss: 7.13854193687439, Validation accuracy: 74.66667175292969
Iteration 20: Loss: 4.883648157119751, Validation accuracy: 77.0
Iteration 30: Loss: 3.252210795879364, Validation accuracy: 85.66667175292969
Iteration 40: Loss: 3.2946232557296753, Validation accuracy: 86.0
Iteration 50: Loss: 2.336420774459839, Validation accuracy: 86.66667175292969
Iteration 60: Loss: 2.233858585357666, Validation accuracy: 87.66667175292969
Iteration 70: Loss: 2.0454314947128296, Validation accuracy: 86.66667175292969
Iteration 80: Loss: 1.9458033442497253, Validation accuracy: 90.0
Iteration 90: Loss: 1.7432460486888885, Validation accuracy: 90.0
Iteration 100: Loss: 2.5129273533821106, Validation accuracy: 86.0
Iteration 110: Loss: 2.4354515075683594, Validation accuracy: 86.33333587646484
Iteration 120: Loss: 1.8821240663528442, Validation accuracy: 89.66667175292969
Iteration 130: Loss: 1.770279735326767, Val

### GoogleNet

In [25]:
googlenet = models.googlenet(pretrained=True)
googlenet

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [26]:
googlenet.fc = nn.Linear(in_features=1024, out_features=3, bias=True)

In [27]:
# Only unfreeze the last two layer
to_update = ["fc.weight", "fc.bias"]
to_update_params = []
for name, param in googlenet.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [28]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [29]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
googlenet = googlenet.to(device)

In [30]:
# Training
googlenet_best_parameters = training(googlenet)

C:\Users\Leo\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Iteration 1: Loss: 8.255078792572021, Validation accuracy: 33.333335876464844
Iteration 10: Loss: 4.824135780334473, Validation accuracy: 46.333335876464844
Iteration 20: Loss: 2.803482711315155, Validation accuracy: 54.0
Iteration 30: Loss: 2.2422669529914856, Validation accuracy: 65.66667175292969
Iteration 40: Loss: 1.803799569606781, Validation accuracy: 77.66667175292969
Iteration 50: Loss: 1.8154978156089783, Validation accuracy: 75.66667175292969
Iteration 60: Loss: 1.8348612785339355, Validation accuracy: 77.33333587646484
Iteration 70: Loss: 1.6584130227565765, Validation accuracy: 78.0
Iteration 80: Loss: 1.7150744199752808, Validation accuracy: 75.66667175292969
Iteration 90: Loss: 1.6798571348190308, Validation accuracy: 77.0
Iteration 100: Loss: 1.7341972589492798, Validation accuracy: 77.0
Iteration 110: Loss: 1.5894401669502258, Validation accuracy: 78.66667175292969
Iteration 120: Loss: 1.5942870378494263, Validation accuracy: 81.33333587646484
Iteration 130: Loss: 1.66

### ResNet

In [31]:
resnet18 = models.resnet18(pretrained=True)
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [32]:
resnet18.fc = nn.Linear(in_features=512, out_features=3, bias=True)

In [33]:
# Only unfreeze the last two layer
to_update = ["fc.weight", "fc.bias"]
to_update_params = []
for name, param in resnet18.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [34]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [35]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

In [36]:
# Training
resnet18_best_parameters = training(resnet18)

Iteration 1: Loss: 12.793030023574829, Validation accuracy: 33.333335876464844
Iteration 10: Loss: 3.8278685808181763, Validation accuracy: 41.66666793823242
Iteration 20: Loss: 2.70078843832016, Validation accuracy: 61.66666793823242
Iteration 30: Loss: 2.1200522780418396, Validation accuracy: 70.0
Iteration 40: Loss: 2.1465656757354736, Validation accuracy: 72.0
Iteration 50: Loss: 2.198509454727173, Validation accuracy: 69.66667175292969
Iteration 60: Loss: 1.7994529008865356, Validation accuracy: 73.66667175292969
Iteration 70: Loss: 1.7965734004974365, Validation accuracy: 77.33333587646484
Iteration 80: Loss: 1.5422567427158356, Validation accuracy: 78.33333587646484
Iteration 90: Loss: 1.4998057186603546, Validation accuracy: 82.0
Iteration 100: Loss: 2.1557387113571167, Validation accuracy: 72.66667175292969
Iteration 110: Loss: 1.711721122264862, Validation accuracy: 77.0
Iteration 120: Loss: 1.3532288670539856, Validation accuracy: 83.0
Iteration 130: Loss: 1.4179449081420898

### MobileNet V3 Small

In [43]:
mobilenet_v3_small = models.mobilenet_v3_small(pretrained=True)
mobilenet_v3_small

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to C:\Users\Leo/.cache\torch\hub\checkpoints\mobilenet_v3_small-047dcff4.pth


  0%|          | 0.00/9.83M [00:00<?, ?B/s]

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride

In [47]:
mobilenet_v3_small.classifier[3] = nn.Linear(in_features=1024, out_features=3, bias=True)

In [48]:
# Only unfreeze the last two layer
to_update = ["classifier.3.weight", "classifier.3.bias"]
to_update_params = []
for name, param in mobilenet_v3_small.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [49]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [50]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilenet_v3_small = mobilenet_v3_small.to(device)

In [51]:
# Training
mobilenet_v3_small_best_parameters = training(mobilenet_v3_small)

Iteration 1: Loss: 2.602459490299225, Validation accuracy: 64.66667175292969
Iteration 10: Loss: 1.624720185995102, Validation accuracy: 79.66667175292969
Iteration 20: Loss: 1.630161851644516, Validation accuracy: 81.33333587646484
Iteration 30: Loss: 1.4899601936340332, Validation accuracy: 86.0
Iteration 40: Loss: 1.3609908819198608, Validation accuracy: 85.0
Iteration 50: Loss: 0.9004408717155457, Validation accuracy: 88.33333587646484
Iteration 60: Loss: 1.0242455899715424, Validation accuracy: 87.33333587646484
Iteration 70: Loss: 0.8408935368061066, Validation accuracy: 89.66667175292969
Iteration 80: Loss: 0.9181053936481476, Validation accuracy: 91.0
Iteration 90: Loss: 0.8077542036771774, Validation accuracy: 89.66667175292969
Iteration 100: Loss: 0.8270334303379059, Validation accuracy: 87.66667175292969
Iteration 110: Loss: 0.8876508176326752, Validation accuracy: 91.0
Iteration 120: Loss: 0.7568303793668747, Validation accuracy: 90.0
Iteration 130: Loss: 0.7719676196575165

### RegNet 16

In [52]:
regnet_x_16gf = models.regnet_x_16gf(pretrained=True)
regnet_x_16gf

Downloading: "https://download.pytorch.org/models/regnet_x_16gf-2007eb11.pth" to C:\Users\Leo/.cache\torch\hub\checkpoints\regnet_x_16gf-2007eb11.pth


  0%|          | 0.00/208M [00:00<?, ?B/s]

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): ConvNormActivation(
          (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): ConvNormActivation(
            (0): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): ConvNormActivation(
            (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=2, bias=False)
            

In [53]:
regnet_x_16gf.fc = nn.Linear(in_features=2048, out_features=3, bias=True)

In [54]:
# Only unfreeze the last two layer
to_update = ["fc.weight", "fc.bias"]
to_update_params = []
for name, param in regnet_x_16gf.named_parameters():
    if name in to_update:
        param.requires_grad = True
        to_update_params.append(param)
    else:
        param.requires_grad = False

In [55]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [56]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
regnet_x_16gf = regnet_x_16gf.to(device)

In [57]:
# Training
regnet_x_16gf_best_parameters = training(regnet_x_16gf)

C:\Users\Leo\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Iteration 1: Loss: 5.210873246192932, Validation accuracy: 26.33333396911621
Iteration 10: Loss: 3.1379557847976685, Validation accuracy: 56.66666793823242
Iteration 20: Loss: 2.3636845350265503, Validation accuracy: 71.33333587646484
Iteration 30: Loss: 1.8639137744903564, Validation accuracy: 78.0
Iteration 40: Loss: 2.1934202909469604, Validation accuracy: 74.0
Iteration 50: Loss: 1.9692952036857605, Validation accuracy: 77.66667175292969
Iteration 60: Loss: 1.7556286454200745, Validation accuracy: 77.0
Iteration 70: Loss: 1.5797682702541351, Validation accuracy: 81.66667175292969
Iteration 80: Loss: 1.3766049146652222, Validation accuracy: 83.33333587646484
Iteration 90: Loss: 1.3521035313606262, Validation accuracy: 84.0
Iteration 100: Loss: 1.780838519334793, Validation accuracy: 78.33333587646484
Iteration 110: Loss: 1.2638126015663147, Validation accuracy: 85.33333587646484
Iteration 120: Loss: 1.0649163872003555, Validation accuracy: 86.33333587646484
Iteration 130: Loss: 1.13

### Expriment on customized CNN model

In [106]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Fully connected output
        self.linear_layers = nn.Sequential(
            nn.Linear(100352, 3)
        )
 
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [107]:
CNNmodel = CNNModel()

In [108]:
CNNmodel

CNNModel(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=3, bias=True)
  )
)

In [109]:
to_update_params = []
for name, param in CNNmodel.named_parameters():
    to_update_params.append(param)

In [110]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [111]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CNNmodel = CNNmodel.to(device)

In [113]:
# Training
CNN_best_parameters = training(CNNmodel)

Iteration 1: Loss: 943.3783721923828, Validation accuracy: 37.66666793823242
Iteration 10: Loss: 276.7934265136719, Validation accuracy: 58.333335876464844
Iteration 20: Loss: 112.2628059387207, Validation accuracy: 56.0
Iteration 30: Loss: 67.32317733764648, Validation accuracy: 55.66666793823242
Iteration 40: Loss: 40.235331535339355, Validation accuracy: 49.0
Iteration 50: Loss: 39.656182289123535, Validation accuracy: 60.66666793823242
Iteration 60: Loss: 36.50485801696777, Validation accuracy: 55.66666793823242
Iteration 70: Loss: 27.05071449279785, Validation accuracy: 62.66666793823242
Iteration 80: Loss: 22.996022701263428, Validation accuracy: 62.66666793823242
Iteration 90: Loss: 16.764676094055176, Validation accuracy: 64.66667175292969
Iteration 100: Loss: 24.290486812591553, Validation accuracy: 63.0
Iteration 110: Loss: 20.393211364746094, Validation accuracy: 68.33333587646484
Iteration 120: Loss: 16.72360134124756, Validation accuracy: 60.333335876464844
Iteration 130: 

In [9]:
class CNNModel2(nn.Module):
    def __init__(self):
        super(CNNModel2, self).__init__()
        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        # Fully connected output
        self.linear_layers = nn.Sequential(
            nn.Linear(100352, 3)
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [10]:
CNNmodel2 = CNNModel2()

In [11]:
CNNmodel2

CNNModel2(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=100352, out_features=3, bias=True)
  )
)

In [12]:
to_update_params = []
for name, param in CNNmodel2.named_parameters():
    to_update_params.append(param)

In [13]:
# Training hyperparameters
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=to_update_params, lr=0.01)

In [14]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CNNmodel2 = CNNmodel2.to(device)

In [15]:
# Training
CNN2_best_parameters = training(CNNmodel2)

C:\Users\Leo\anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Iteration 1: Loss: 1148.8886413574219, Validation accuracy: 33.333335876464844
Iteration 10: Loss: 365.78282165527344, Validation accuracy: 44.333335876464844
Iteration 20: Loss: 175.01366806030273, Validation accuracy: 49.333335876464844
Iteration 30: Loss: 133.0359115600586, Validation accuracy: 54.0
Iteration 40: Loss: 79.01835441589355, Validation accuracy: 52.0
Iteration 50: Loss: 37.67836952209473, Validation accuracy: 55.333335876464844
Iteration 60: Loss: 23.879998683929443, Validation accuracy: 59.333335876464844
Iteration 70: Loss: 20.642870903015137, Validation accuracy: 62.0
Iteration 80: Loss: 14.403226852416992, Validation accuracy: 63.66666793823242
Iteration 90: Loss: 17.358800888061523, Validation accuracy: 64.33333587646484
Iteration 100: Loss: 22.848021507263184, Validation accuracy: 53.333335876464844
Iteration 110: Loss: 17.597300052642822, Validation accuracy: 62.66666793823242
Iteration 120: Loss: 9.655442714691162, Validation accuracy: 69.66667175292969
Iteratio